# Протоколы аутентификации с нулевым разглашением

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел
from Crypto.Util import number

# КГПСЧ
from Crypto.Random import random

# Понадобится в протоколе Шнорра
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Hash import SHA256

# Немного математики
import gmpy2

# Для генерации идентификаторов клиентов
import uuid

---

In [4]:
def randint(a, b) -> gmpy2.mpz:
    """
    Генерация случайного числа с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_a = int(a)
    native_b = int(b)
    
    return gmpy2.mpz(random.randint(native_a, native_b))

def getrandbits(n) -> gmpy2.mpz:
    """
    """
    
    native_n = int(n)
    
    return gmpy2.mpz(random.getrandbits(native_n))

----

## Протокол Фиата-Шамира
![Fiat-Shamir Protocol](./images/Fiat-Shamir_2.png)

In [5]:
class FiatShamirCA(object):
    PRIME_BITS = 1024
    
    def __init__(self):
        """
        Инициализация доверенного центра.
        Генерируются два простых числа и их произведение.
        """
        
        self._db = {}
        
        self._p = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._q = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._modulus = gmpy2.mul(self._p, self._q)
        
        trace('[FiatShamirCA]', f'Modulus = {self._modulus}')
        
        
    def register_client(self, client_id: uuid.UUID, client_public_key: gmpy2.mpz) -> None:
        """
        Регистрация клиента. На вход принимает идентификатор клиента и его открытый ключ.
        Ничего не возвращает.
        Выбрасывает исключение, если клиент уже существует.
        """
        
        trace('[FiatShamirCA]', f'''Attempting to register client {client_id} 
        with public key {client_public_key}''')
        
        if client_id in self._db:
            trace('[FiatShamirCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Теперь сохраняю открытый ключ
        #
        
        self._db[client_id] = client_public_key
        trace('[FiatShamirCA]', 'Client registered successfully')
        
        
    def get_public_key(self, client_id: uuid.UUID) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента. Получает на вход идентификатор клиента.
        Возвращает открытый ключ.
        Выбрасывает исключение, если пользователь отсутствует.
        """
        
        return self._db[client_id]
       
        
    @property
    def modulus(self) -> gmpy2.mpz:
        """
        Возвращает модуль.
        """
        
        return self._modulus

In [6]:
class FiatShamirClient(object):
    def __init__(self, modulus: gmpy2.mpz, *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz] = None):
        """
        Инициализация клиента с выработкой ключевой пары и идентификатора.
        Последний параметр требуются для демонстрации неудачной аутентификации.
        """
        
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        self._r = None
        
        self._private_key = FiatShamirClient._generate_coprime(modulus)
        self._public_key = gmpy2.powmod(self._private_key, 2, modulus) if fake_data is None else fake_data[1]
        
        trace('[FiatShamirClient]', f'Client {self._id}')
        trace('[FiatShamirClient]', f'Public key = {self._public_key}')
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора.
        """
        
        return self._id
    
    
    def get_x(self, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение случайного значения, вычисляемого по формуле:
                    x = r ** 2 mod n,
        где n - модуль, r - случайное целое число из отрезка [1, n - 1].
        Значение r сохраняется.
        """
        
        self._r = randint(1, modulus - 1)
        return gmpy2.powmod(self._r, 2, modulus)
    
    
    def get_y(self, e: gmpy2.mpz, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение числа, вычисляемого по формуле:
                    y = r * s ** e mod n,
        где n - модуль, s - закрытый ключ пользователя, r - сгенерированное 
        функцией get_x внутреннее значение, e - целое число из отрезка [0, 1],
        пришедшее от проверяющего.
        """
        
        product = gmpy2.mul(self._r, gmpy2.powmod(self._private_key, e, modulus))
        return gmpy2.t_mod(product, modulus)
        
        
    @staticmethod
    def _generate_coprime(modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Генерация случайного числа из отрезка [1, n - 1], взаимно
        простого с n, где n - модуль.
        """
        
        #
        # Генерирую число до тех пор, пока оно не станет взаимно простым с модулем
        #
        
        result = randint(1, modulus - 1)
        while gmpy2.gcd(result, modulus) != 1:
            result = randint(1, modulus - 1)
            
        return result

In [7]:
class FiatShamirVerifier(object):
    def __init__(self, iterations: int):
        """
        Инициализация проверяющего. Задает коичество итераций проверки.
        """
        
        self._iterations = iterations
        
        
    def authenticate(self, client: FiatShamirClient, ca: FiatShamirCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        trace('[FiatShamirVerifier]', f'Attempt to authenticate client {client.identifier}')
        trace('[FiatShamirVerifier]', f'Number of iterations = {self._iterations}')
        
        is_successful = True
        
        for i in range(self._iterations):
            x = client.get_x(ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {x = }')
            
            e = randint(0, 1)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {e = }')
            
            y = client.get_y(e, ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {y = }')
            
            pk = ca.get_public_key(client.identifier)
            product = gmpy2.mul(x, gmpy2.powmod(pk, e, ca.modulus))
            
            successful_iteration = gmpy2.powmod(y, 2, ca.modulus) == gmpy2.t_mod(product, ca.modulus)
            is_successful &= successful_iteration
            
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] success: {successful_iteration}')
            
            if not successful_iteration:
                break
            
        print(f'Authentication successful: {is_successful}')
        return is_successful

----

In [8]:
# Создаю доверенный центр
ca = FiatShamirCA()

[TRACE] [FiatShamirCA] Modulus = 12757231888174565009077970008122194184016282576577131176528301070097635254812254106119227405298877349225630532066245512638898020726524922851989124640457313618685806822274480541556637611112003311630387240082166367888889998372336064372423363519468657246841036840676796125275049754428003879190299871153547812931609026613452894992623836108029293506465393113772580858961794548923253513702478337274729228192418319135016629537133013237722256727484892263649801420087577239431231518377946839987491214953267725026465053794548667298027523299569094819180503513678162178646431707868033206087409700322259258240302733918012214802379


In [9]:
# Создаю двух пользователей:
# - Алису - валидного пользователя
# - Еву - пользователя, представляющегося Алисой. но не знающего закрытого ключа Алисы
alice = FiatShamirClient(ca.modulus)
eve = FiatShamirClient(ca.modulus, 
                       fake_data=(alice.identifier, alice.public_key))

[TRACE] [FiatShamirClient] Client f9d38d17-ec2e-4916-bcdd-fdd8aa78d2e4
[TRACE] [FiatShamirClient] Public key = 3424795262031102549951868746033702516575501634215812503313946557819023111914663095708894439301803781400594126577843131814143987558660306891703643471195248239828026558428344736728225123902128112812983865760409512250154132585610281437457149688076011787264768385351862646817178790179357400901466984043525311547419122105657813245072562765149045598252731438558650618608181395296967702470751413555701784709094942424741438058825005497979134831223930971600902377354091011646874277169535375131808131398942795601383961354264406242028406210705185964757364197387982339493605264168963796911592000299325202482851242080230213446091
[TRACE] [FiatShamirClient] Client f9d38d17-ec2e-4916-bcdd-fdd8aa78d2e4
[TRACE] [FiatShamirClient] Public key = 34247952620311025499518687460337025165755016342158125033139465578190231119146630957088944393018037814005941265778431318141439875586603068917036434711952482398280

In [10]:
# Регистрирую валидного пользователя
ca.register_client(alice.identifier, alice.public_key)

[TRACE] [FiatShamirCA] Attempting to register client f9d38d17-ec2e-4916-bcdd-fdd8aa78d2e4 
        with public key 3424795262031102549951868746033702516575501634215812503313946557819023111914663095708894439301803781400594126577843131814143987558660306891703643471195248239828026558428344736728225123902128112812983865760409512250154132585610281437457149688076011787264768385351862646817178790179357400901466984043525311547419122105657813245072562765149045598252731438558650618608181395296967702470751413555701784709094942424741438058825005497979134831223930971600902377354091011646874277169535375131808131398942795601383961354264406242028406210705185964757364197387982339493605264168963796911592000299325202482851242080230213446091
[TRACE] [FiatShamirCA] Client registered successfully


In [11]:
# Для простоты будет 3 итерации
ITERATIONS = 3

# Создаю проверяющую сторону
verifier = FiatShamirVerifier(ITERATIONS)

In [12]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client f9d38d17-ec2e-4916-bcdd-fdd8aa78d2e4
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(1259123927643860793117901579115497483724101032265979692599789425844766868009465840499176459628774461657822028520176455099541111062125894131357137426140398388008739773809114922461168405489684644573933169445139690629447515453315680144150010237507880514300088690521926099529722597989842974102815554490849853761223532495275307588484178055069472780691692278778766208117361115424299409431051330349935967110367453538733051420429394147346742645738086834958180224798221917472121074529129256869381470800412411966629954179586086284102497168060880545145425714608263485244840469138426306104644485187542682746881526660979500728010)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(0)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(495887603511213443814492824844719765051334241244603986284302548643442378794

True

In [13]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client f9d38d17-ec2e-4916-bcdd-fdd8aa78d2e4
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(6499430496506233576841874790089790722632743764356799364808482499888339720078099409627799622255739086464308570236975443530160954224260273177361404777018008875234728232858046478380441416513744038482311212977623982471013858306887833012338963178851394167644351850566350194035923537291408895472306586033885725900380616719378472174412971993678655137037171372548657348761824253832218686939009346608159744889969074798942630673052507264958108346896571181353606039112859743691169912266753190825572610913925365683612147009709977033509510549594335217450867453447621620733964719014159652678423198975436505510103876269597175224271)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(1)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(133997779557307295786016754033799567635110101501616647248602260056853471543

False

---

## Протокол Шнорра
![Schnorr Protocol](./images/Schnorr_2.png)

In [14]:
class SchnorrCA(object):
    RSA_KEY_LENGTH = 4096
    
    def __init__(self):
        """
        Инициализация доверенного центра
        Генерация простых чисел p, q и элемента g, а также ключевой пары СА
        """
        
        self._db = {}
        
        #
        # Генерация параметров протокола
        #
        
        self._p, self._q, self._g = SchnorrCA._generate_parameters()
        
        #
        # Ключевая пара RSA для СА
        #
        
        self._private_key = RSA.generate(SchnorrCA.RSA_KEY_LENGTH)
        self._public_key = self._private_key.publickey()
        
        trace('[SchnorrCA]', f'RSA public key = {(self._public_key.e, self._public_key.n)}')
        
        
    def register_client(self, 
                        client_id: uuid.UUID, 
                        client_public_key: gmpy2.mpz) -> tuple[uuid.UUID, gmpy2.mpz, bytes]:
        """
        Регистрация клиента. Принимает на вход идентификатор клиента и его открытый ключ.
        Возвращает подписанный сервером сертификат.
        Выбрасывает исключение, если клиент уже зарегистрирован.
        """
        
        if client_id in self._db:
            trace('[SchnorrCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Создаю сертификат
        #
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        signature = pss.new(self._private_key).sign(h)
        
        certificate = (client_id, client_public_key, signature)
        trace('[SchnorrCA]', f'Certificate = {certificate}')
        
        #
        # Сохраняю его и возвращаю
        #
        
        self._db[client_id] = certificate
        return certificate
    
    
    def verify_certificate(self, certificate: tuple[uuid.UUID, gmpy2.mpz, bytes]) -> bool:
        """
        Проверка сертификата.
        Возвращает признак валидности сертификата.
        """
        
        client_id, client_public_key, signature = certificate
        
        if client_id not in self._db:
            trace('[SchnorrCA]', f"Client {client_id} doesn't exist")
            raise ValueError("Client doesn't exist")
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        verifier = pss.new(self.public_key)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа СА.
        """
        
        return self._public_key
    
    
    @property
    def parameters(self) -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Параметры протокола (p, q, g)
        """
        
        return self._p, self._q, self._g
        
        
    @staticmethod
    def _generate_parameters() -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Генерация параметров протокола (p, q, g).
        """
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = gmpy2.mpz(number.getPrime(Q_LENGTH))
        p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
        
        while not gmpy2.is_prime(p):
            p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
            
        #
        # Сгенерирую число, взаимно простое с p
        #
        
        tmp = getrandbits(P_LENGTH)
        while gmpy2.gcd(tmp, p) != 1:
            tmp = getrandbits(P_LENGTH)
        
        #
        # По малой теореме Ферма такой g будет иметь порядок q
        #
        
        g = gmpy2.powmod(tmp, (p - 1) // q, p)
        
        trace('[SchnorrCA]', f'{p = }, {q = }, {g = }')
        return p, q, g

In [15]:
class SchnorrClient(object):
    def __init__(self, parameters: tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz], *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz, tuple] = None):
        """
        Инициализация клиента. Генерирует идентификатор клиента,
        ключевую пару.
        Последний параметр необходим для демонстрации неуспешной аутентификации.
        """
        
        self._p, self._q, self._g = parameters
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        
        self._cert = None if fake_data is None else fake_data[2]
        self._r = None
        
        self._private_key = randint(1, self._q - 1)
        self._public_key  = gmpy2.powmod(
            self._g, -self._private_key, self._p) if fake_data is None else fake_data[1]
        
        trace('[SchnorrClient]', f'Client {self._id}')
        trace('[SchnorrClient]', f'Public key = {self._public_key}')
        
        
    def register(self, ca: SchnorrCA) -> None:
        """
        Регистрация клиента на СА. Принимает дескриптор СА.
        """
        
        self._cert = ca.register_client(self.identifier, self.public_key)
        
        
    def get_x(self) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = g ** r mod p
        где r - случайное целое число из отрезка [1, q - 1].
        Сохраняет значение r.
        """
        
        self._r = randint(1, self._q - 1)
        return gmpy2.powmod(self._g, self._r, self._p)
    
    
    def get_y(self, e, security_parameter) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = a * e + r mod q
        где r - сохраненное в get_x служебное значение, e - число,
        полученное от проверяющего, a - закрытый ключ клиента.
        """
        
        if e > 2 ** security_parameter or e < 1:
            trace('[SchnorrClient]', f'Invalid {e = }')
            raise ValueError('Invalid e')
            
        pre_result = gmpy2.mul(self._private_key, e) + self._r
        return gmpy2.t_mod(pre_result, self._q)
    
    
    @property
    def certificate(self):
        """
        Получение сертификата клиента.
        """
        
        return self._cert
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента.
        """
        
        return self._public_key
        
        
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора клиента.
        """
        
        return self._id

In [16]:
class SchnorrVerifier(object):
    def __init__(self, parameters, security_parameter):
        """
        Инициализация проверяющего. Фактически только закрепляет параметры протокола.
        """
        
        self._p, self._q, self._g = parameters
        self._security_parameter = security_parameter
        
        assert security_parameter > 0 and 2 ** security_parameter < self._q
        
        self._e_values = set()
        
        
    def authenticate(self, client: SchnorrClient, ca: SchnorrCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        #
        # Получаю с клиента x и его сертификат
        #
        
        x, cert = client.get_x(), client.certificate
        trace('[SchnorrVerifier]', f'{x = }')
        
        #
        # Если сертификат битый или невалидный, то это плохо
        #
        
        if not ca.verify_certificate(cert):
            trace('[SchnorrVerifier]', f'Invalid certificate {cert}')
            return False
            
        #
        # Генерирую число е и отдаю его клиенту
        #
            
        e = randint(1, 2 ** self._security_parameter)
        while e in self._e_values:
            e = randint(1, 2 ** self._security_parameter)
            
        self._e_values.add(e)        
        trace('[SchnorrVerifier]', f'{e = }')
        
        y = client.get_y(e, self._security_parameter)
        trace('[SchnorrVerifier]', f'{y = }')
        
        #
        # Вычисляю значение, которое необходимо сравнить с полученным от клиента х
        # И сразу же сравниваю их
        # 
        
        product = gmpy2.mul(gmpy2.powmod(self._g, y, self._p), 
                            gmpy2.powmod(client.public_key, e, self._p))
        z = gmpy2.t_mod(product, self._p)
        trace('[SchnorrVerifier]', f'{z = }')
        
        is_successful = x == z
        print(f'Authentication successful: {is_successful}')
        
        return is_successful

----

In [17]:
# Создаю доверенный центр
ca = SchnorrCA()

[TRACE] [SchnorrCA] p = mpz(47725458810204906958783227026245876429099147152447119178906889486804043670133184041942433065216169018810422826554707581097632325576477261010249548551100934015332987472420336609877787282731250872950443063057069492780849630593280459721656650307775162664122367226499868559342730127690247531485563208750696432641), q = mpz(1111490627038773470224248058634461914892728789353), g = mpz(25665120655039498108453345959671637957374758918757107549964083041660088565381364594017780814743458955960947189375400609210306153040317267363290622963149048626541291797717125400569724831766216180134049647922702816860533095503316404823949844094982065762581911856953265711125010298361872752789358573349941691063)
[TRACE] [SchnorrCA] RSA public key = (65537, 7701181669725416794717983794083315137340640023133025670542384227969808613322966932082946899114786116168378781997745003785138958552855094432473785028850661712258925504498638626508796481644765384651936833310972271608067045338268986340419978

In [18]:
# Создам Алису - валидного пользователя
alice = SchnorrClient(ca.parameters)

[TRACE] [SchnorrClient] Client 04b147fd-877d-46dc-91c6-63522beca9ad
[TRACE] [SchnorrClient] Public key = 8315321307818975939897820533648926130721652386045056034736103843005193759278525774454528428097317750908140513307085274843953465292010932488834901299881993612777708630617621263583481334444362106228650441502437119820451840169459074253471545334129982519056191300159604861223291951040390233626259783939972930317


In [19]:
# Регистрирую валидного пользователя
alice.register(ca)

[TRACE] [SchnorrCA] Certificate = (UUID('04b147fd-877d-46dc-91c6-63522beca9ad'), mpz(8315321307818975939897820533648926130721652386045056034736103843005193759278525774454528428097317750908140513307085274843953465292010932488834901299881993612777708630617621263583481334444362106228650441502437119820451840169459074253471545334129982519056191300159604861223291951040390233626259783939972930317), b'\n\x15O\xae\xeenB\xb4\xcc\xe5{\xe5\x0e\xf2\xd9\x0b^O\x93\xa00\x16\xaeU\xe5DK\x1c7\xd0U\xbb\x82Mlw\xd7\xb2j\xd1\x93\x0cg\xd3\xf1E\xeeo=hU\xfb\xedR]\xd6;[\x86\xd1V\x05\xb5v\x96C\xa9n\xf1\xa6\xce\x95\xad\xabh\xd6g\xbe\xa2\xee\x06Q\xd3\x81"\x80[\xb0`\x85k\x92\xd8\xa1\xd8\xf7x\x87\x16<L\xb1]La\xdb\x9c\x14i\xab\x0b\xafk\x1aB#Rf\x92\xe3\r\xc8\xa6_\xabz\xc1\x9d\x99\xbdY\xdc\xfc?\xe9\xf4B\\|\xa4\xd58\xbf\r\x7fx\xc5\xc0\xa6\x84\xb2\xdb\xc3\xecjSK\x05h\tkU\xd11s,\xa4\xdcy\xa76O\xd8k\xed{\x03i\'\xeb\x11\xee\xe6\x04\xf6l\xd9\xac\xcf\xc0\'m\xb0\xf8i\xaf1\xfeQ\x7f\xe6\x17\x85*\xc5\xcd\x9c\x07B~\x895\x80\x1f^R\x

In [20]:
# Параметр безопасности положим равным 10 для демонстрации
security_parameter = 10

# Создаю проверяющую сторону
verifier = SchnorrVerifier(ca.parameters, security_parameter)

In [21]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [SchnorrVerifier] x = mpz(5659365052950006972198928207659603123770232549782131287911271169370255164644056413713996677453720679374531350739365884737133673548931924499051169662983733994701022486349105985437076139858327240781243464715291020816282309467839150337057471134140769575490360653555349718661221655931558599664241475867635461131)
[TRACE] [SchnorrVerifier] e = mpz(435)
[TRACE] [SchnorrVerifier] y = mpz(617054535075182030181485007472405347280869749025)
[TRACE] [SchnorrVerifier] z = mpz(5659365052950006972198928207659603123770232549782131287911271169370255164644056413713996677453720679374531350739365884737133673548931924499051169662983733994701022486349105985437076139858327240781243464715291020816282309467839150337057471134140769575490360653555349718661221655931558599664241475867635461131)
Authentication successful: True


True

In [22]:
# Создаю Еву - клиента, который пытается представиться Алисой
eve = SchnorrClient(ca.parameters, 
                    fake_data=(alice.identifier, alice.public_key, alice.certificate))

[TRACE] [SchnorrClient] Client 04b147fd-877d-46dc-91c6-63522beca9ad
[TRACE] [SchnorrClient] Public key = 8315321307818975939897820533648926130721652386045056034736103843005193759278525774454528428097317750908140513307085274843953465292010932488834901299881993612777708630617621263583481334444362106228650441502437119820451840169459074253471545334129982519056191300159604861223291951040390233626259783939972930317


In [23]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [SchnorrVerifier] x = mpz(41092458591976667786022839805306240413003337480088362818915516235098800195864185350448357387718608683139791587989559806967637886605498342945979606929372130728013926824730381026238733580738101731698979870916415125305543417090534953346639954364378814536856671732727052225926658742378520439012325400246139039372)
[TRACE] [SchnorrVerifier] e = mpz(874)
[TRACE] [SchnorrVerifier] y = mpz(742717080023826812045439628157802904666414547115)
[TRACE] [SchnorrVerifier] z = mpz(8870221455270082928152806869301801236525268717544624479624464731604909105398435874297625870715299763476961518137292628865623967441527867120617833637840286466182105266180201568149409260361782005398127183095119247214546038735773055806590754354332621971919608651039653031845686441150761525470881612092918814344)
Authentication successful: False


False